# Packages

In [11]:
import numpy as np
import pandas as pd
from scipy.special import expit  # Sigmoid function for logistic regression

from dowhy import CausalModel
import dowhy.causal_estimators.generalized_linear_model_estimator
import dowhy.datasets
import dowhy.causal_estimators.linear_regression_estimator
import statsmodels.api as sm
import statsmodels.formula.api as smf

import dowhy.causal_estimators.linear_regression_estimator
import statsmodels.genmod.families.links as links

# Warnings and logging
import warnings
warnings.filterwarnings('ignore')

# Data Simulation

In [215]:
# Simulation parameters
np.random.seed(42)
n = 10000  # Number of samples
beta_M0 = 1 # Intercept for mediator model
beta_MX = 2  # Effect of promotion on time spent reading reviews
beta_Y0 = 1.0  # Intercept for outcome model
beta_YX = 2.5  # Direct effect of promotion on buying product A
beta_YM = -1.2  # Effect of time spent reading reviews on buying product A
sigma_M = 1.0  # Standard deviation of mediator noise
beta_income_X = 1.5  # Effect of income on promotion
beta_income_Y = 1.5  # Effect of income on buying product A

# Simulate confounder (income), continuous
income = np.random.normal(2, 0.5, n)

# Simulate treatment (promotion), binary, influenced by income
promotion_prob = expit(beta_income_X * income)  # Probability of promotion depends on income
promotion = np.random.binomial(1, promotion_prob, n)

# Simulate mediator (time spent reading reviews), continuous
time_spent_reading_reviews = beta_M0 + beta_MX * promotion  + np.random.normal(0, sigma_M, n)

# Simulate outcome (bought_product_A), binary
linear_predictor_Y = (
    beta_Y0
    + beta_YX * promotion
    + beta_YM * time_spent_reading_reviews
    + beta_income_Y * income
)
bought_product_A = np.random.binomial(1, expit(linear_predictor_Y))

# Collider depends on promotion and outcome
total_amount_spent = 5 + 3 * promotion + 20 * bought_product_A + np.random.normal(5, 10, n)


# Create DataFrame
data = pd.DataFrame({
    'promotion': promotion,
    'time_spent_reading_reviews': time_spent_reading_reviews,
    'bought_product_A': bought_product_A,
    'income': income,
    'total_amount_spent': total_amount_spent
})


data.mean()

promotion                      0.936300
time_spent_reading_reviews     2.868374
bought_product_A               0.903200
income                         1.998932
total_amount_spent            31.014293
dtype: float64

# Different ways to analyze the same data

In [162]:
graph="""digraph {
promotion -> bought_product_A;
promotion -> time_spent_reading_reviews;
time_spent_reading_reviews -> bought_product_A;
income -> bought_product_A;
income -> promotion;
income -> time_spent_reading_reviews;
}
"""

## Mediation analysis

In [203]:
model = CausalModel(data,
                    treatment = 'promotion',
                    outcome = 'bought_product_A',
                    mediator='time_spent_reading_reviews',
                    common_causes=['income'],
                    identify_vars=True,
                    graph=graph,
                  )


# Natural direct effect (nde)
identified_estimand_nde = model.identify_effect(estimand_type="nonparametric-nde",
                                            proceed_when_unidentifiable=False)
print(identified_estimand_nde)

# Natural indirect effect (nie)
identified_estimand_nie = model.identify_effect(estimand_type="nonparametric-nie",
                                            proceed_when_unidentifiable=False)
print(identified_estimand_nie)

Estimand type: nonparametric-nde

### Estimand : 1
Estimand name: mediation
Estimand expression:
Expectation(Derivative(bought_product_A|time_spent_reading_reviews, [promotion
]))
Estimand assumption 1, Mediation: time_spent_reading_reviews intercepts (blocks) all directed paths from promotion to b,o,u,g,h,t,_,p,r,o,d,u,c,t,_,A except the path {promotion}→{b,o,u,g,h,t,_,p,r,o,d,u,c,t,_,A}.
Estimand assumption 2, First-stage-unconfoundedness: If U→{promotion} and U→{time_spent_reading_reviews} then P(time_spent_reading_reviews|promotion,U) = P(time_spent_reading_reviews|promotion)
Estimand assumption 3, Second-stage-unconfoundedness: If U→{time_spent_reading_reviews} and U→bought_product_A then P(bought_product_A|time_spent_reading_reviews, promotion, U) = P(bought_product_A|time_spent_reading_reviews, promotion)

Estimand type: nonparametric-nie

### Estimand : 1
Estimand name: mediation
Estimand expression:
Expectation(Derivative(bought_product_A, [time_spent_reading_reviews])*Derivat

In [204]:
probit = links.probit()
causal_estimate_nie = model.estimate_effect(identified_estimand_nie,
                                        method_name="mediation.two_stage_regression",
                                       confidence_intervals=False,
                                       test_significance=False,
                                        method_params = {
                                            'first_stage_model': dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator,
                                            'second_stage_model': dowhy.causal_estimators.generalized_linear_model_estimator.GeneralizedLinearModelEstimator,
                                            'glm_family': sm.families.Binomial(probit),
                                        }
                                       )
print(causal_estimate_nie)

causal_estimate_nde = model.estimate_effect(identified_estimand_nde,
                                        method_name="mediation.two_stage_regression",
                                       confidence_intervals=False,
                                       test_significance=False,
                                        method_params = {
                                            'first_stage_model': dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator,
                                            'second_stage_model': dowhy.causal_estimators.generalized_linear_model_estimator.GeneralizedLinearModelEstimator,
                                            'glm_family': sm.families.Binomial(probit),
                                        }
                                       )
print(causal_estimate_nde)

two_stage_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': False, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': [], 'first_stage_model': <class 'dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator'>, 'second_stage_model': <class 'dowhy.causal_estimators.generalized_linear_model_estimator.GeneralizedLinearModelEstimator'>, 'glm_family': <statsmodels.genmod.families.family.Binomial object at 0x000001EF570D2D48>}
*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-nie

### Estimand : 1
Estimand name: mediation
Estimand expression:
Expectation(Derivative(bought_product_A, [time_spent_reading_reviews])*Derivat
ive([time_spent_reading_reviews], [promotion]))
Estimand assumption 1, Mediation: time_spent_reading_reviews intercepts (blocks) all directed paths from promotion to b,o,u,g,h,t,_,p,r,o,d,u,c,t,_,A except the path {promotion}→{b,o,u,g,h,t,_,p,r,o,

In [205]:
(causal_estimate_nde.value + causal_estimate_nie.value) * 100

## Adjusting for confounder

In [216]:
# Predict probabilities for promotion = 0 and promotion = 1
data_no_promotion = data.copy()
data_no_promotion['promotion'] = 0  # Set promotion to 0
data_promotion = data.copy()
data_promotion['promotion'] = 1  # Set promotion to 1

# Fit the simpler model
model = sm.GLM.from_formula("bought_product_A ~ promotion + income", data, 
                                 family=sm.families.Binomial(link=sm.families.links.probit()))
mdl = model.fit()

# Predict probabilities
prob_no_promotion = mdl.predict(data_no_promotion)
prob_promotion = mdl.predict(data_promotion)

# Marginal effect on probability scale
me_correct = np.mean(prob_promotion - prob_no_promotion)
print(me_correct*100)
mdl.summary()

-1.5407006144638307


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       bought_product_A   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                 probit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3004.6
Date:                Mon, 09 Dec 2024   Deviance:                       6009.2
Time:                        10:29:21   Pearson chi2:                 1.01e+04
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1062      0.087      1.221      0.222      -0.064       0.277
promotion     -0.1001      0.070     -1.436      0.151      -0.237       0.036
income         0.6801      0.038     17.898      0.000       0.606       0.755
==============================================================================
"""

The mediation analysis and adjusting for confounder is the correct way to analyze the effect. Mediation analysis will add the benefit of separating direct and indirect effects, which can be very helpful. In this case we can see that reading the reviews is really detrimental for the effectiveness of the campaign.

## Adjusting for confounder + collider

In [218]:
# Predict probabilities for promotion = 0 and promotion = 1
data_no_promotion = data.copy()
data_no_promotion['promotion'] = 0  # Set promotion to 0
data_promotion = data.copy()
data_promotion['promotion'] = 1  # Set promotion to 1

# Fit the simpler model
model = sm.GLM.from_formula("bought_product_A ~ promotion + income + total_amount_spent", data, 
                                 family=sm.families.Binomial(link=sm.families.links.probit()))
mdl = model.fit()

# Predict probabilities
prob_no_promotion = mdl.predict(data_no_promotion)
prob_promotion = mdl.predict(data_promotion)

# Marginal effect on probability scale
me_con_col = np.mean(prob_promotion - prob_no_promotion)
print(me_con_col*100)
mdl.summary()

-3.3364042018227464


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       bought_product_A   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9996
Model Family:                Binomial   Df Model:                            3
Link Function:                 probit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1615.4
Date:                Mon, 09 Dec 2024   Deviance:                       3230.8
Time:                        10:29:45   Pearson chi2:                 1.16e+04
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -2.2327      0.131    -17.019      0.000      -2.490      -1.976
promotion             -0.4291      0.095     -4.537      0.000      -0.615      -0.244
income                 0.7084      0.052     13.581      0.000       0.606       0.811
total_amount_spent     0.1112      0.003     37.819      0.000       0.105       0.117
======================================================================================
"""

## Adjusting for everything

In [224]:
# Predict probabilities for promotion = 0 and promotion = 1
data_no_promotion = data.copy()
data_no_promotion['promotion'] = 0  # Set promotion to 0
data_promotion = data.copy()
data_promotion['promotion'] = 1  # Set promotion to 1

# Fit the simpler model
model = sm.GLM.from_formula("bought_product_A ~ promotion + income + total_amount_spent + time_spent_reading_reviews", data, 
                                 family=sm.families.Binomial(link=sm.families.links.probit()))
mdl = model.fit()

# Predict probabilities
prob_no_promotion = mdl.predict(data_no_promotion)
prob_promotion = mdl.predict(data_promotion)

# Marginal effect on probability scale
me_all = np.mean(prob_promotion - prob_no_promotion)
print(me_all*100)
mdl.summary()

6.3313454455849545


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       bought_product_A   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9995
Model Family:                Binomial   Df Model:                            4
Link Function:                 probit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1393.0
Date:                Mon, 09 Dec 2024   Deviance:                       2785.9
Time:                        10:59:16   Pearson chi2:                 1.40e+04
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     -1.5859      0.143    -11.090      0.000      -1.866      -1.306
promotion                      0.6736      0.115      5.838      0.000       0.447       0.900
income                         0.8097      0.057     14.114      0.000       0.697       0.922
total_amount_spent             0.1102      0.003     35.167      0.000       0.104       0.116
time_spent_reading_reviews    -0.5704      0.030    -19.319      0.000      -0.628      -0.513
==============================================================================================
"""

## No adjustments

In [219]:
# Predict probabilities for promotion = 0 and promotion = 1
data_no_promotion = data.copy()
data_no_promotion['promotion'] = 0  # Set promotion to 0
data_promotion = data.copy()
data_promotion['promotion'] = 1  # Set promotion to 1

# Fit the simpler model
model = sm.GLM.from_formula("bought_product_A ~ promotion", data, 
                                 family=sm.families.Binomial(link=sm.families.links.probit()))
mdl = model.fit()

# Predict probabilities
prob_no_promotion = mdl.predict(data_no_promotion)
prob_promotion = mdl.predict(data_promotion)

# Marginal effect on probability scale
me_con_col = np.mean(prob_promotion - prob_no_promotion)
print(me_con_col*100)
mdl.summary()

2.9070637941421893


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       bought_product_A   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                Binomial   Df Model:                            1
Link Function:                 probit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3177.3
Date:                Mon, 09 Dec 2024   Deviance:                       6354.5
Time:                        10:30:10   Pearson chi2:                 1.00e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1551      0.064     18.108      0.000       1.030       1.280
promotion      0.1558      0.066      2.351      0.019       0.026       0.286
==============================================================================
"""

In [222]:
# Same results as doing no adjustments
data.groupby('promotion').agg('bought_product_A').mean().diff()[1]